In [1]:
!pip install surprise
!pip install scikit-surprise

In [2]:
%%time
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix, coo_matrix
from tqdm.auto import tqdm
from collections import Counter
from surprise import Dataset, Reader
import time
from surprise import KNNBaseline

CPU times: total: 328 ms
Wall time: 317 ms


c:\python38\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df_path = r"C:\Users\dlbol\Downloads\likes\likes\likes_data\track_artists.csv"
train_path = r"C:\Users\dlbol\Downloads\likes\likes\likes_data\train"
test_path = r"C:\Users\dlbol\Downloads\likes\likes\likes_data\test"
pred_path = r"C:\Users\dlbol\Downloads\likes\likes\likes_data\surprize_artist_KNNBaseline_target"
target_path = r"C:\Users\dlbol\Downloads\likes\likes\likes_data\surprize_artist_KNNBaseline_pred"

In [4]:
def calc_mrr(predict, answer):
    for i in range(len(predict)):
        if predict[i] == answer:
            return 1. / (i + 1)
    return 0

max_prediction_len = 100

def calc_score(target_path, predict_path):
    with open(target_path) as f:
        y_true = [int(x.strip()) for x in f.readlines()]

    with open(predict_path) as f:
        y_pred = [[int(x) for x in line.strip().split(' ')] for line in f.readlines()]

    mrr_score = 0
    for (pred, answer) in zip(y_pred, y_true):
        if len(pred) > max_prediction_len:
            raise ValueError('$maximum prediction length is {}, got {}$'.format(max_prediction_len, len(y_pred[i])))
        mrr_score += calc_mrr(pred, answer)

    print(f"MRR@100 = {(mrr_score / len(y_true)):.4f}")

In [5]:
%%time
df = pd.read_csv(df_path)
df.index = df['trackId']

CPU times: total: 62.5 ms
Wall time: 63.8 ms


In [6]:
%%time

list_coord = []
target_val = []
np.random.seed(42)  
val_id = np.random.choice(range(10000), size=1000, replace=False)
val_id.sort()
all_artist = set()
ls_artists = []

with open(train_path) as f:
    lines = f.readlines()
    idx = 0 
    for line in tqdm(lines):
        tracks = line.strip().split(' ')
        tracks_arr = np.array(tracks)
        tracks_arr = tracks_arr.astype(np.int32)
        artist_arr = np.array([df.at[track, 'artistId'] for track in tracks_arr])
        if idx in val_id:
            target_val.append(artist_arr[-1])
            artist_arr = artist_arr[:-1]
        ls_artists.extend(artist_arr)
        all_artist = all_artist.union(set(artist_arr))
        artist_cnt = Counter(artist_arr)
        list_coord_user = [(idx, i[0], i[1]) for i in artist_cnt.items()]
        list_coord.extend(list_coord_user)
        idx += 1
        if idx > 10000:
            break

  1%|▌                                                                       | 10000/1160084 [00:09<18:36, 1029.65it/s]

CPU times: total: 10.8 s
Wall time: 10.8 s


In [7]:
%%time

data = pd.DataFrame(list_coord, columns=['uid', 'iid', 'rating_row'])
data['rating_max'] = data.rating_row / data.groupby('uid')['rating_row'].transform('max')
data['rating_part'] = data.rating_row / data.groupby('uid')['rating_row'].transform('sum')

CPU times: total: 2.91 s
Wall time: 2.92 s


# Baseline

In [8]:
all_artist_cnt = [i[0] for i in Counter(ls_artists).most_common(100)]

result = [' '.join(map(str, all_artist_cnt)) + '\n' for _ in val_id]

with open(pred_path, 'w') as f:
    f.writelines(result)

with open(target_path, 'w') as f:
    for i in target_val:
        f.write(f'{i}\n')

calc_score(target_path, pred_path)

MRR@100 = 0.0144


# RATING_MAX

In [9]:
%%time

reader = Reader(rating_scale=(0, 1))
dataset = Dataset.load_from_df(data[['uid', 'iid', 'rating_max']], reader)
trainset = dataset.build_full_trainset()

CPU times: total: 672 ms
Wall time: 682 ms


## KNNBaseline

In [10]:
def optimize(k=40, 
             min_k=1,
             user_based=True,
             name="pearson_baseline",
             verbose=True):

    start_time = time.time()
    
    sim_options = {
        "name": name,
        "user_based": user_based
    }
    
    algo = KNNBaseline(k=k, min_k=min_k, sim_options=sim_options, verbose=True)
    
    algo.fit(trainset)
    
    print("--- fit: %s seconds ---" % (time.time() - start_time))
    
    pred = []
    for uid in tqdm(val_id):
        pred_user = []
        for iid in all_artist:
            score = algo.predict(uid=uid, iid=iid).est
            pred_user.append((iid, score))
        pred.append(pred_user)

    res = []
    for user in tqdm(pred):
        ls = sorted(user, key=lambda x: x[1])
        temp = [i[0] for i in ls[-100:]]
        temp.reverse()
        res.append(temp)

    result = [' '.join(map(str, i)) + '\n' for i in res]


    with open(pred_path, 'w') as f:
        f.writelines(result)

    with open(target_path, 'w') as f:
        for i in target_val:
            f.write(f'{i}\n')

    calc_score(target_path, pred_path)

In [11]:
for name in ['msd', 'pearson', 'pearson_baseline']:
    for k in [10, 40, 80]:
        for min_k in [1, 10, 50]:
            print(f'name: {name}, k: {k}, min_k: {min_k}')
            optimize(k=k, 
                     min_k=min_k,
                     user_based=True,
                     name=name,
                     verbose=True)

name: msd, k: 10, min_k: 1
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
--- fit: 8.614154815673828 seconds ---


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:11<00:00, 86.16it/s]


MRR@100 = 0.0057
name: msd, k: 10, min_k: 10
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
--- fit: 8.680148363113403 seconds ---


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:10<00:00, 99.29it/s]


MRR@100 = 0.0797
name: msd, k: 10, min_k: 50
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
--- fit: 8.286507844924927 seconds ---


...

MRR@100 = 0.0095
name: msd, k: 80, min_k: 1
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
--- fit: 7.668088912963867 seconds ---


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:12<00:00, 79.21it/s]


MRR@100 = 0.0021
name: msd, k: 80, min_k: 10
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
--- fit: 8.692912817001343 seconds ---


 17%|█████████████▏                                                                 | 167/1000 [02:14<11:08,  1.25it/s]


KeyboardInterrupt: 

In [13]:
for name in ['msd', 'pearson', 'pearson_baseline']:
    print(f'name: {name}')
    optimize(k=40, 
             min_k=10,
             user_based=True,
             name=name,
             verbose=True)

name: msd
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
--- fit: 8.848728895187378 seconds ---


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:10<00:00, 91.50it/s]


MRR@100 = 0.0721
name: pearson
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
--- fit: 15.268238544464111 seconds ---


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:10<00:00, 95.85it/s]


MRR@100 = 0.0566
name: pearson_baseline
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
--- fit: 13.353877067565918 seconds ---


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:08<00:00, 113.17it/s]


MRR@100 = 0.0699


# RATING_PART

In [14]:
%%time

reader = Reader(rating_scale=(0, 1))
dataset = Dataset.load_from_df(data[['uid', 'iid', 'rating_part']], reader)
trainset = dataset.build_full_trainset()

CPU times: total: 734 ms
Wall time: 746 ms


In [15]:
optimize(k=40, 
         min_k=10,
         user_based=True,
         name='msd',
         verbose=True)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
--- fit: 8.510480165481567 seconds ---


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:07<00:00, 125.08it/s]


MRR@100 = 0.0280


In [16]:
optimize(k=40, 
         min_k=10,
         user_based=True,
         name='pearson_baseline',
         verbose=True)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
--- fit: 11.22592830657959 seconds ---


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:07<00:00, 129.45it/s]


MRR@100 = 0.0341


## Best

In [17]:
%%time

reader = Reader(rating_scale=(0, 1))
dataset = Dataset.load_from_df(data[['uid', 'iid', 'rating_max']], reader)
trainset = dataset.build_full_trainset()

CPU times: total: 828 ms
Wall time: 837 ms


In [18]:
for k in [30, 40, 50]:
    for min_k in [5, 10, 20]:
        print(f'k: {k}, min_k: {min_k}')
        optimize(k=k, 
                 min_k=min_k,
                 user_based=True,
                 name='msd',
                 verbose=True)

k: 30, min_k: 5
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
--- fit: 7.960039138793945 seconds ---


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:08<00:00, 113.26it/s]


MRR@100 = 0.0692
k: 30, min_k: 10
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
--- fit: 7.453402519226074 seconds ---


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:08<00:00, 119.45it/s]


MRR@100 = 0.0740
k: 30, min_k: 20
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
--- fit: 7.628616571426392 seconds ---


...

MRR@100 = 0.0615
k: 50, min_k: 10
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
--- fit: 14.37860369682312 seconds ---


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:10<00:00, 94.11it/s]


MRR@100 = 0.0696
k: 50, min_k: 20
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
--- fit: 8.087398290634155 seconds ---


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:08<00:00, 114.85it/s]

MRR@100 = 0.0725


In [19]:
for k in [25, 30, 35]:
    for min_k in [20, 30, 50]:
        print(f'k: {k}, min_k: {min_k}')
        optimize(k=k, 
                 min_k=min_k,
                 user_based=True,
                 name='msd',
                 verbose=True)

k: 25, min_k: 20
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
--- fit: 8.972921371459961 seconds ---


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:16<00:00, 61.11it/s]


MRR@100 = 0.0773
k: 25, min_k: 30
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
--- fit: 12.591449499130249 seconds ---


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:07<00:00, 127.51it/s]


MRR@100 = 0.0095
k: 25, min_k: 50
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
--- fit: 7.331418752670288 seconds ---


...

MRR@100 = 0.0751
k: 35, min_k: 30
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
--- fit: 7.3092734813690186 seconds ---


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:08<00:00, 121.96it/s]


MRR@100 = 0.0754
k: 35, min_k: 50
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
--- fit: 7.1944260597229 seconds ---


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:07<00:00, 126.99it/s]


MRR@100 = 0.0095


## Final best
k: 25, min_k: 20<br>
Estimating biases using als...<br>
Computing the msd similarity matrix...<br>
Done computing similarity matrix.<br>
--- fit: 8.972921371459961 seconds ---<br>
100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [10:58<00:00,  1.52it/s]<br>
100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:16<00:00, 61.11it/s]<br>
MRR@100 = 0.0773